#**DTL Pipeline**

##**Create Streaming Table**

###Reading Data From Silver Layer

In [0]:
from pyspark.sql.functions import col

The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-8773269880400616>, line 1
----> 1 import dlt
      2 from  pyspark.sql.functions import *
      3 from pyspark.sql.types import *

ModuleNotFoundError: No module named 'dlt'

In [0]:
import dlt

my_rule = {
    "rule1": "ProductID is not null",
    "rule2": "ProductName is not null",
    "rule3": "Category is not null"
}

@dlt.table()
@dlt.expect_all_or_drop(my_rule)
def Dimproduct_stage():
    df = spark.readStream.table("databricks_ecommerce.silver.product")
    return df

### Creating Streaming View At Top pf Streaming Table - As transitional Layer - Streaming View


In [0]:

@dlt.view()
def Dimproduct_view():

    df = spark.readStream.table("Live.Dimproduct_stage")

    return df



### Create Empty DimProduct - Empty Container to Desposit Final Value

In [0]:
dlt.create_streaming_table("Dimproducts")


### By Using Apply_Change_api- We Will Upsert Value in Empty Target Table

In [0]:
dlt.apply_changes(
    target="Dimproducts",
    source="Live.Dimproduct_view",
    keys=["ProductID"],
    sequence_by=col("LaunchDate"),   # ensure LaunchDate exists and is timestamp/date
    stored_as_scd_type=2
)